In [ ]:
import psycopg2 as ps
from wordcloud import WordCloud
import  matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import re, sys
import numpy as np

sys.path.append('./TweetData')
from TweetData import keys, Preprocessing

import pandas as pd
import json
from collections import Counter
from fuzzywuzzy import fuzz, process

import geopandas as gpd

In [ ]:
# geo_df = gpd.GeoDataFrame.from_features(
#     px.data.election_geojson()["features"]
# ).merge(df, on="district").set_index("district")
places1 = places.set_index('osm_id')

fig = px.choropleth_mapbox(places1,
            geojson=places1.way,
            locations=places1.index,
            # color="Joly",
            mapbox_style="open-street-map",
)
fig.show()

In [ ]:
## Clean text WordCloud
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()
q = '''SELECT TWEET_ID, TWEET_TEXT FROM TWEETS ORDER BY TWEET_CREATED DESC '''
cursor.execute(q)
tweets = cursor.fetchall()
cursor.close()
conn.close()

tweets = pd.DataFrame(tweets, columns=['id', 'text'])

In [ ]:
## Clean text WordCloud
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
# cursor = conn.cursor()
q = '''SELECT OSM_ID, NAME, WAY FROM PLACES '''
# cursor.execute(q)
# places = cursor.fetchall()
places = gpd.GeoDataFrame.from_postgis(q, conn, geom_col='way')
# cursor.close()
# conn.close()
# places = pd.DataFrame(places, columns=['id', 'name', 'geom'])

In [ ]:
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
q1 = '''SELECT GID, NOMBRE, GEOM FROM carreteras WHERE NOMBRE IS NOT NULL '''
# cursor.execute(q)
# places = cursor.fetchall()
roads = gpd.GeoDataFrame.from_postgis(q1, conn, geom_col='geom')
# cursor.close()
# conn.close()
# places = pd.DataFrame(places, columns=['id', 'name', 'geom'])

In [ ]:
places = places[places.name.str.len()>5]

In [ ]:
extract = Preprocessing.clean_text(tweets, 'text')
# tweets.clean.str.findall('([A-Z]\S+)')
# re.findall('([A-Z]\S+)', tweets)
extract['possible_place'] = extract.clean.str.findall('([A-Z]\S+)')

In [ ]:
extract = extract[extract.clean.str.contains('accident.*|cho[qc].*|colision.*|vuelco|volcó|vuelca|atropell.*|tran[qc].*|embotella.*|trafico|huelga|protest.*|trabajo.*|cierr.*|detenid.*|obstacul.*|incendi.*|inunda.*|lluvia')]
extract = extract[extract.clean.str.len()>30]
extract = extract[extract.possible_place.notnull()]

In [ ]:
test = 'tranque pesado en la Via Brasil hacia el interior'
matches1 = [fuzz.token_sort_ratio(place, test) for place in places.name]
# sum(matches1)
matches2 = [fuzz.token_set_ratio(place, test) for place in places.name]
# max(matches2)

In [ ]:
process.extract(test, places, scorer=fuzz.token_set_ratio)

In [ ]:
extract['contains'] = extract['clean'].apply(lambda x: np.array([]))
# extract['contains'] = extract['contains'].astype(object)
extract['possible'] = extract['clean'].apply(lambda x: np.array([]))
# extract['possible'] = extract['possible'].astype(object)

In [ ]:
extract.drop(columns=['contains', 'possible'], inplace=True)

In [ ]:
extract2 = extract.copy()

In [250]:
def get_road(text, roads):
    choice = process.extractOne(text, roads.nombre, scorer=fuzz.token_set_ratio)
    if choice[1]>=80:
        # Returns the gid
        return str(roads.iloc[roads.index==choice[2], 0].iloc[0])
    else: return ''

In [251]:
def get_place(text, places):
    choice = process.extractOne(text, places.name, scorer=fuzz.token_set_ratio)
    if choice[1]>=80:
        # Returns the osm_id
        return str(places.iloc[places.index==choice[2], 0].iloc[0])
    else: return ''

In [ ]:
places.head(1)

In [256]:
extract2 = extract[100:500]

In [257]:
extract2.loc[:, 'road'] = extract2.apply(lambda row: get_road(row.clean, roads), axis=1)

C:\Users\lucia\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\lucia\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [258]:
extract2.loc[:, 'place'] = extract2.apply(lambda row: get_place(row.clean, places), axis=1)

C:\Users\lucia\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\lucia\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [259]:
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()
ins = '''INSERT INTO TWEETS_PLACES(TWEET_ID, ROAD_GID, PLACE_ID)
    VALUES (%s, %s, %s) '''
for index, row in extract2.iterrows():
    cursor.execute(ins, 
        (row['id'], None if row['road']=='' else row['road'], 
            None if row['place']=='' else row['place']))
    conn.commit()
cursor.close()
conn.close()

In [254]:
extract2

,id,text,clean,possible_place,road,place
0,1401376450134089731,Se registra accidente de tránsito en La Vía In...,registra accidente transito Via Interamericana...,"[Via, Interamericana, Coronado]",,384384811
2,1401343653847650304,[VÍDEO] Se registra accidente de tránsito en L...,registra accidente transito Autopista Panama C...,"[Autopista, Panama, Colon, Chilibre]",,
4,1401322507232231424,"Vistas del tráfico en la Vía Interamericana, p...",vistas trafico Via Interamericana Rod Carew di...,"[Via, Interamericana, Rod, Carew, Panama, Oeste]",,384384811
6,1401320841711923200,[VÍDEO] Se registra colisión en la Avenida Ric...,registra colision Avenida Ricardo J Alfaro San...,"[Avenida, Ricardo, Alfaro, San, Miguel, Arcang...",2192,
8,1401302282440949765,Panamá el país donde te la tienes que rifar en...,panama pais te tienes rifar calles puedes caer...,"[Estadio, Andy, Alonso]",,
...,...,...,...,...,...,...
151,1400490929245638661,[VÍDEO] Se registra colisión en la Avenida Ric...,registra colision Avenida Ricardo J Alfaro Pla...,"[Avenida, Ricardo, Alfaro, Plaza, Edison, Pana...",2192,72197504
153,1400486702083448835,"Mula detenida en la Vía Centenario, próximo al...",mula detenida Via Centenario Rod Carew direcci...,"[Via, Centenario, Rod, Carew, Panama, Oeste]",,385466258
155,1400461460661170196,Se mantiene el tráfico afectado en ambas direc...,mantiene trafico afectado ambas direcciones Vi...,"[Via, Transistmica, Villa]",,563131085
156,1400451986076536834,🅾️ Se registra accidente de tránsito en La Vía...,registra accidente transito Via Interamericana...,"[Via, Interamericana, Anclas, Mall, Chorrera, ...",,731137347


In [ ]:
extract1 = extract.copy()

In [ ]:
for index, tweet in extract[:100].iterrows():
    intweet = []; possible = []
    for index_place, place in places.iterrows():
        if len(place['name'])>5:
            if place['name'] in tweet['clean']:
                intweet.append(place['osm_id'])
                # extract.iloc[index, 'contains'] = extract.iloc[index, 'contains'].append
            elif (fuzz.token_set_ratio(place['name'], tweet['clean'])==100 and fuzz.token_sort_ratio(place['name'], tweet['clean'])>50) or (fuzz.token_set_ratio(place['name'], tweet['clean'])>90):
                possible.append(place['osm_id'])
                # print('\n', tweet, ' - possible - ', place)
            #     if fuzz.token_sort_ratio(place, tweet)>50:
            #         possible.append(place.name)
            #         # print('\n\t', tweet, ' - possible %i - ' % fuzz.token_sort_ratio(place, tweet), place)
            # elif fuzz.token_set_ratio(place, tweet)>90:
            #     possible.append(place.name)
                # print('\n', tweet, '  - similar %i to -  ' % fuzz.token_set_ratio(place, tweet), place)
    if len(intweet)>0:
        extract.iloc[index, 4] = np.array(intweet)
    if len(possible)>0:
        extract.iloc[index, 5] = np.array(possible)

In [ ]:
extract[extract.possible>0]